In [1]:
import glob
import itertools
import pandas as pd

In [2]:
orig_files = glob.glob('../rita_data_ibk/orig_files/*.xlsx')[1:]

['../rita_data_ibk/orig_files\\lg_michaelsburg.xlsx',
 '../rita_data_ibk/orig_files\\sg_bruneck.xlsx']

In [3]:
dfs = []
for file in orig_files:
    keys = list(pd.read_excel(file, None).keys())
    for x in keys:
        df = pd.read_excel(file, x)
        dfs.append(df)

In [4]:
inst, _ = Institution.objects.get_or_create(written_name="Südtiroler Landesarchiv")

In [5]:
def create_entry(row, repo_name='Südtiroler Landesarchiv'):
    signatur = row['Signatur']
    vb = signatur.split(',')[0]
    year = vb[-4:]
    vb_sig = "{}, {}".format(repo_name, vb)
    vb_obj, _ = VerfachBuch.objects.get_or_create(signatur=vb_sig)
    inv, _ = InventoryEntry.objects.get_or_create(inv_signatur="{}, {}".format(repo_name, signatur))
    inv.is_located_in = vb_obj
    inv.save()
    return inv
    

In [47]:
def relate_person(row, rel_type, persons, entry):
    for y in persons:
        if y.startswith(')'):
            pers = None
        elif y.startswith('[Vorstehe'):
            pers = None
        elif y.startswith('Im Rahm'):
            pers = None
        elif y.startswith('[die Personen'):
            pers = None
        elif y.startswith('n.a'):
            pers = None
        else:
            try:
                pers, _ = Person.objects.get_or_create(written_name=y)
            except Exception as e:
                pers = None
        if pers:
            getattr(entry, rel_type, None).add(pers)
    return entry
            
       

In [52]:
for x in dfs:
    for i, row in x.iterrows():
        entry = create_entry(row)
#         persons = str(row['Beteiligte Personen (Beamte, Gerichtsverpflichtete, Zeugen, ...)']).split('\n')
#         relate_person(row, 'adm_person', persons, entry)
        persons = str(row['Genannte Personen']).split('\n')
        relate_person(row, 'other_person', persons, entry)